In [29]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Embedding, SpatialDropout1D, LSTM, BatchNormalization, Dropout, Dense, CuDNNLSTM

In [9]:
from UtilityFunctions import *

In [10]:
Labels = ['country', 'id', 'title', 'c1', 'c2', 'c3', 'description', 'price', 'type']

In [11]:
train_df = pd.read_csv("data_train.csv", names=Labels)
test_df = pd.read_csv("data_test.csv", names=Labels)

In [12]:
PreProcess(train_df)
PreProcess(test_df)

C:\Users\uzair\anaconda3\lib\site-packages\bs4\__init__.py:332: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\uzair\anaconda3\lib\site-packages\bs4\__init__.py:332: MarkupResemblesLocatorWarning: "....." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [13]:
train_df = train_df.drop(['country','id','price','type'], axis=1)
test_df = test_df.drop(['country','id','price','type'], axis=1)

In [14]:
train_df['title'] = train_df['title'] + ' ' + train_df['description']
test_df['title'] = test_df['title'] + ' ' +test_df['description']

In [15]:
train_df = train_df.drop(['description'], axis=1)
test_df = test_df.drop(['description'], axis=1)

In [16]:
train_df['c3'] = train_df['c3'].replace(np.nan, 'none', regex=True)
test_df['c3'] = test_df['c3'].replace(np.nan, 'none', regex=True)

In [17]:
train_df

title  \
0      adana gallery suri square hijab light pink mat...   
1      cuba heartbreaker eau de parfum spray ml oz fo...   
2      andoer cm cellphone smartphone mini dualheaded...   
3      anmyna complaint silky set shampoo ml conditio...   
4      argital argiltubo green clay for face and body...   
...                                                  ...   
36278  sades led backlit wired usb mechanical gaming ...   
36279  sona electric oven seo years warranty thermal ...   
36280  op portable wireless bluetooth speaker with ha...   
36281  wootwoot tictactoe pillow case white cotton sa...   
36282  new smart wristband smart bracelet heart rate ...   

                                          c1                    c2  \
0                                    Fashion                 Women   
1                            Health & Beauty           Bath & Body   
2      TV, Audio / Video, Gaming & Wearables                 Audio   
3                            Health & Beauty             Hair Care   
4                            Health & Beauty            Men's Care   
...                                      ...                   ...   
36278                    Computers & Laptops  Computer Accessories   
36279                        Home Appliances      Large Appliances   
36280                    Computers & Laptops  Computer Accessories   
36281                          Home & Living               Bedding   
36282  TV, Audio / Video, Gaming & Wearables   Wearable Technology   

                            c3  
0                  Muslim Wear  
1             Hand & Foot Care  
2           Live Sound & Stage  
3      Shampoos & Conditioners  
4           Body and Skin Care  
...                        ...  
36278                Keyboards  
36279       Microwaves & Ovens  
36280                 Speakers  
36281       Pillows & Bolsters  
36282        Activity Trackers  

[36283 rows x 4 columns]

# Create padded input sequences

In [18]:
token = RegexpTokenizer("[\w']+")
maxWords = 20000
MaxWordLength = train_df.title.map(lambda x: len(token.tokenize(x))).max()
tokenizer = Tokenizer(num_words = maxWords, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train_df['title'].values)
VocabSize = len(tokenizer.word_index)

In [19]:
MaxWordLength

856

In [20]:
VocabSize

57408

### Train input data sequences

In [21]:
train_X_c1 = tokenizer.texts_to_sequences((train_df['title']).values)
train_X_c1 = tf.keras.preprocessing.sequence.pad_sequences(train_X_c1, maxlen=MaxWordLength)

print('Shape of data tensor:', train_X_c1.shape)


train_X_c2 = tokenizer.texts_to_sequences((train_df['title']+' '+train_df['c1']).values)
train_X_c2 = tf.keras.preprocessing.sequence.pad_sequences(train_X_c2, maxlen=MaxWordLength)

print('Shape of data tensor:', train_X_c2.shape)


train_X_c3 = tokenizer.texts_to_sequences((train_df['title']+' '+train_df['c1']+' '+train_df['c2']).values)
train_X_c3 = tf.keras.preprocessing.sequence.pad_sequences(train_X_c3, maxlen=MaxWordLength)

print('Shape of data tensor:', train_X_c3.shape)

Shape of data tensor: (36283, 856)
Shape of data tensor: (36283, 856)
Shape of data tensor: (36283, 856)


## Test input data sequences

In [22]:
test_X_c1 = tokenizer.texts_to_sequences((test_df['title']).values)
test_X_c1 = tf.keras.preprocessing.sequence.pad_sequences(test_X_c1, maxlen=MaxWordLength)

print('Shape of test data tensor:', test_X_c1.shape)


test_X_c2 = tokenizer.texts_to_sequences((test_df['title']+' '+test_df['c1']).values)
test_X_c2 = tf.keras.preprocessing.sequence.pad_sequences(test_X_c2, maxlen=MaxWordLength)

print('Shape of test data tensor:', test_X_c2.shape)


test_X_c3 = tokenizer.texts_to_sequences((test_df['title']+' '+test_df['c1']+' '+test_df['c2']).values)
test_X_c3 = tf.keras.preprocessing.sequence.pad_sequences(test_X_c3, maxlen=MaxWordLength)

print('Shape of test data tensor:', test_X_c3.shape)


Shape of test data tensor: (11838, 856)
Shape of test data tensor: (11838, 856)
Shape of test data tensor: (11838, 856)


# Y-label Encoding

## Train label encoding

In [23]:
train_Y_c1 = pd.get_dummies(train_df['c1']).values
print('Shape of label tensor:', train_Y_c1.shape)

train_Y_c2 = pd.get_dummies(train_df['c2']).values 
print('Shape of label tensor:', train_Y_c2.shape)

train_Y_c3 = pd.get_dummies(train_df['c3']).values
print('Shape of label tensor:', train_Y_c3.shape)

Shape of label tensor: (36283, 9)
Shape of label tensor: (36283, 57)
Shape of label tensor: (36283, 185)


## Test label encoding

In [24]:
test_Y_c1 = pd.get_dummies(test_df['c1']).values
print('Shape: ', test_Y_c1.shape)

test_Y_c2 = pd.get_dummies(test_df['c2']).values
print('Shape: ', test_Y_c2.shape)

test_Y_c3 = pd.get_dummies(test_df['c3']).values
print('Shape: ', test_Y_c3.shape)

Shape of test label tensor: (11838, 9)
Shape of test label tensor: (11838, 57)
Shape of test label tensor: (11838, 185)


In [25]:
labels_c1=sorted(train_df['c1'].unique())
labels_c2=sorted(train_df['c2'].unique())
labels_c3=sorted(train_df['c3'].unique())

# Creating RNN Architecture and Train Model - Category1

In [19]:
# c1_count = train_df['c1'].nunique()
# embeddingDimensions = 100

# model_c1 = Sequential()
# model_c1.add(Embedding(VocabSize, embeddingDimensions, input_length = MaxWordLength))
# model_c1.add(LSTM(512, activation="tanh", recurrent_activation="sigmoid", use_bias=True, unroll=False))
# model_c1.add(Dropout(0.4))
# model_c1.add(Dense(128, activation='relu'))
# model_c1.add(Dense(c1_count, activation = 'softmax'))
# model_c1.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# model_c1.summary()

In [35]:
# epochs = 30
# batch_size = 128

# history1 = model_c1.fit(train_X_c1, train_Y_c1,
#                     validation_data=(test_X_c1, test_Y_c1),
#                     epochs = epochs, batch_size = batch_size,
#                     shuffle = True,
#                     callbacks = [ModelCheckpoint('h_models_rnn/model_c1/{val_accuracy:.3f}', save_weights_only=False)])

Epoch 1/30
284/284 [==============================] - 79s 266ms/step - loss: 1.5439 - accuracy: 0.4393 - val_loss: 0.4105 - val_accuracy: 0.8773


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c1\0.877\assets


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c1\0.877\assets


Epoch 2/30
284/284 [==============================] - 76s 267ms/step - loss: 0.3398 - accuracy: 0.9041 - val_loss: 0.3801 - val_accuracy: 0.8952


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c1\0.895\assets


...

INFO:tensorflow:Assets written to: h_models_rnn_td/model_c1\0.912\assets


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c1\0.912\assets


Epoch 30/30
284/284 [==============================] - 73s 257ms/step - loss: 0.0078 - accuracy: 0.9979 - val_loss: 0.6806 - val_accuracy: 0.9062


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c1\0.906\assets


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c1\0.906\assets


# Creating RNN Architecture and Train Model - Category2

In [38]:
# c2_count=train_df['c2'].nunique()
# embeddingDimensions = 100

# model_c2 = Sequential()
# model_c2.add(Embedding(VocabSize, embeddingDimensions, input_length = MaxWordLength))
# model_c2.add(LSTM(512, activation="tanh", recurrent_activation="sigmoid", use_bias=True, unroll=False))
# model_c2.add(Dropout(0.4))
# model_c2.add(Dense(128, activation='relu'))
# model_c2.add(Dense(c2_count, activation = 'softmax'))
# model_c2.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# model_c2.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 856, 100)          5740800   
_________________________________________________________________
lstm_6 (LSTM)                (None, 512)               1255424   
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_13 (Dense)             (None, 57)                7353      
Total params: 7,069,241
Trainable params: 7,069,241
Non-trainable params: 0
_________________________________________________________________


In [39]:
# epochs = 30
# batch_size = 128

# history2 = model_c2.fit(train_X_c2, train_Y_c2,
#                     validation_data=(test_X_c2, test_Y_c2),
#                     epochs = epochs, batch_size = batch_size,
#                     shuffle = True,
#                     callbacks = [ModelCheckpoint('h_models_rnn/model_c2/{val_accuracy:.3f}', save_weights_only=False)])

Epoch 1/30
284/284 [==============================] - 77s 262ms/step - loss: 2.5034 - accuracy: 0.3811 - val_loss: 0.7944 - val_accuracy: 0.7431


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c2\0.743\assets


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c2\0.743\assets


Epoch 2/30
284/284 [==============================] - 74s 260ms/step - loss: 0.6831 - accuracy: 0.7786 - val_loss: 0.6210 - val_accuracy: 0.7929


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c2\0.793\assets


...

INFO:tensorflow:Assets written to: h_models_rnn_td/model_c2\0.872\assets


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c2\0.872\assets


Epoch 30/30
284/284 [==============================] - 76s 267ms/step - loss: 0.0138 - accuracy: 0.9965 - val_loss: 0.9663 - val_accuracy: 0.8711


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c2\0.871\assets


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c2\0.871\assets


# Creating RNN Architecture and Train Model - Category3

In [40]:
# c3_count=train_df['c3'].nunique()
# embeddingDimensions = 100

# model_c3 = Sequential()
# model_c3.add(Embedding(VocabSize, embeddingDimensions, input_length = MaxWordLength))
# model_c3.add(LSTM(512, activation="tanh", recurrent_activation="sigmoid", use_bias=True, unroll=False))
# model_c3.add(Dropout(0.4))
# model_c3.add(Dense(256, activation='relu'))
# model_c3.add(Dense(c3_count, activation = 'softmax'))
# model_c3.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# model_c3.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 856, 100)          5740800   
_________________________________________________________________
lstm_7 (LSTM)                (None, 512)               1255424   
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_15 (Dense)             (None, 185)               47545     
Total params: 7,175,097
Trainable params: 7,175,097
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# epochs = 30
# batch_size = 128

# history3 = model_c3.fit(train_X_c3, train_Y_c3,
#                     validation_data=(test_X_c3, test_Y_c3),
#                     epochs = epochs, batch_size = batch_size,
#                     shuffle = True,
#                     callbacks = [ModelCheckpoint('h_models_rnn/model_c3/{val_accuracy:.3f}', save_weights_only=False)])

Epoch 1/30
284/284 [==============================] - 79s 267ms/step - loss: 3.5131 - accuracy: 0.2630 - val_loss: 1.1731 - val_accuracy: 0.5834


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c3\0.583\assets


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c3\0.583\assets


Epoch 2/30
284/284 [==============================] - 75s 265ms/step - loss: 1.0470 - accuracy: 0.6147 - val_loss: 0.8845 - val_accuracy: 0.6617


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c3\0.662\assets


...

INFO:tensorflow:Assets written to: h_models_rnn_td/model_c3\0.787\assets


Epoch 29/30
284/284 [==============================] - 78s 273ms/step - loss: 0.0267 - accuracy: 0.9908 - val_loss: 1.6467 - val_accuracy: 0.7907


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c3\0.791\assets


INFO:tensorflow:Assets written to: h_models_rnn_td/model_c3\0.791\assets


Epoch 30/30
284/284 [==============================] - ETA: 0s - loss: 0.0275 - accuracy: 0.9913

# Finding The Model With Best Validation Accuracy

In [2]:
l = glob.glob('h_models_rnn/model_c1/*')
l = [format(float(os.path.basename(x)), '.3f') for x in l]
m1 = max(l)

In [3]:
l = glob.glob('h_models_rnn/model_c2/*')
l = [format(float(os.path.basename(x)), '.3f') for x in l]
m2 = max(l)

In [4]:
l = glob.glob('h_models_rnn/model_c3/*')
l = [format(float(os.path.basename(x)), '.3f') for x in l]
m3 = max(l)

# Load Models

In [5]:
model_c1=tf.keras.models.load_model('h_models_rnn/model_c1/{}'.format(m1))

In [6]:
model_c2=tf.keras.models.load_model('h_models_rnn/model_c2/{}'.format(m2))

In [7]:
model_c3=tf.keras.models.load_model('h_models_rnn/model_c3/{}'.format(m3))

# Model Evaluation

In [26]:
acc = model_c1.evaluate(test_X_c1, test_Y_c1)
ModelAccuracy1 = acc[1]
print('ModelAccuracy for c1 =',acc[1]*100,'%')

370/370 [==============================] - 1362s 4s/step - loss: 0.3114 - accuracy: 0.9219
ModelAccuracy for c1 = 92.18618273735046 %


In [27]:
acc = model_c2.evaluate(test_X_c2, test_Y_c2)
ModelAccuracy2 = acc[1]
print('ModelAccuracy for c2 =',acc[1]*100,'%')

370/370 [==============================] - 1132s 3s/step - loss: 0.4968 - accuracy: 0.8811
ModelAccuracy for c2 = 88.10610175132751 %


In [28]:
acc = model_c3.evaluate(test_X_c3, test_Y_c3)
ModelAccuracy3 = acc[1]
print('ModelAccuracy for c3 =',acc[1]*100,'%')

370/370 [==============================] - 1110s 3s/step - loss: 0.8648 - accuracy: 0.7995
ModelAccuracy for c3 = 79.95438575744629 %


# Testing Query

In [40]:
data = {'title':[''],
        'description':['']}

query_df = pd.DataFrame(data)

query_df['title'] = str(input("Enter the product title : "))

PreProcess(query_df)

query_df['title'] = query_df['title'] + ' ' + query_df['description']

query_df = query_df.drop(['description'], axis=1)

seq = tokenizer.texts_to_sequences((query_df['title']).values)
padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=MaxWordLength)

pred1 = model_c1.predict(padded)
print('Category 1: ',labels_c1[np.argmax(pred1)])

query_df['title'] = query_df['title'] + ' ' + labels_c1[np.argmax(pred1)]

seq = tokenizer.texts_to_sequences((query_df['title']).values)
padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=MaxWordLength)

pred2 = model_c2.predict(padded)
print('Category 2: ',labels_c2[np.argmax(pred2)])

query_df['title'] = query_df['title'] +' '+ labels_c2[np.argmax(pred2)]

seq = tokenizer.texts_to_sequences((query_df['title']).values)
padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=MaxWordLength)

pred3 = model_c3.predict(padded)
print('Category 3: ',labels_c3[np.argmax(pred3)])

Enter the product title : sad
Category 1:  Fashion
Category 2:  Women
Category 3:  Muslim Wear
